In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
d = '18-04-2022'
df = pd.read_csv(f'./Worldometer-data/table_{d}_raw.csv')
df.drop(columns='Unnamed: 0', inplace=True)
df.head()

,"Country, Other",Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,New Recovered,Active Cases,"Serious, Critical",Tot Cases/1M pop,Deaths/1M pop,Total Tests,Tests/1M pop,Population
0,World,"505,263,460","+355,765","6,225,557","+1,483","456,606,790","+1,014,825","42,431,113","42,014","64,821",798.7,NaN,NaN,NaN
1,China,"185,035","+2,742","4,641",+3,"151,407","+1,637","28,987",71,129,3,"160,000,000","111,163","1,439,323,776"
2,USA,"82,383,494","+26,881","1,015,821",+109,"80,244,093","+41,383","1,123,580","1,438","246,299","3,037","996,639,125","2,979,620","334,485,290"
3,India,"43,045,527","+1,247","521,996",+31,"42,511,701",+928,"11,830",698,"30,652",372,"832,104,846","592,521","1,404,347,026"
4,Brazil,"30,261,088","+8,470","662,076",+65,"29,262,483","+35,432","336,529","8,318","140,572","3,076","63,776,166","296,261","215,270,453"


- Rename `Country, Other` thành `Country`

In [3]:
df.rename(columns={'Country, Other': 'Country', 'Serious, Critical': 'Serious'}, inplace=True)
df.head()

,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,New Recovered,Active Cases,Serious,Tot Cases/1M pop,Deaths/1M pop,Total Tests,Tests/1M pop,Population
0,World,"505,263,460","+355,765","6,225,557","+1,483","456,606,790","+1,014,825","42,431,113","42,014","64,821",798.7,NaN,NaN,NaN
1,China,"185,035","+2,742","4,641",+3,"151,407","+1,637","28,987",71,129,3,"160,000,000","111,163","1,439,323,776"
2,USA,"82,383,494","+26,881","1,015,821",+109,"80,244,093","+41,383","1,123,580","1,438","246,299","3,037","996,639,125","2,979,620","334,485,290"
3,India,"43,045,527","+1,247","521,996",+31,"42,511,701",+928,"11,830",698,"30,652",372,"832,104,846","592,521","1,404,347,026"
4,Brazil,"30,261,088","+8,470","662,076",+65,"29,262,483","+35,432","336,529","8,318","140,572","3,076","63,776,166","296,261","215,270,453"


## Bỏ dữ liệu thừa


### Bỏ các dòng thừa
- Trong dữ liệu gốc, cột `Country` có bao gồm 2 thành phần không phải là country: ['Diamond Princess', 'MS Zaandam'], số liệu từ những thành phần này thiếu rất nhiều và không có đóng góp nhiều chi việc trực quan hóa lúc sau, do đó chúng nên bị bỏ 
- `Country` là 'World' (dòng đầu trong `df`) và 'Total' (dòng cuối trong `df`). Thực chất 2 dòng này có thể tính được thông qua việc lấy tổng các thuộc tính 

In [4]:
del_cols = ['Diamond Princess', 'MS Zaandam', 'World', 'Total:']

In [5]:
print(f'no. of countries (prev): {len(df)}')
df = df[~df['Country'].isin(del_cols)]
print(f'no. of countries (now): {len(df)}')
df.head()

no. of countries (prev): 230
no. of countries (now): 226


,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,New Recovered,Active Cases,Serious,Tot Cases/1M pop,Deaths/1M pop,Total Tests,Tests/1M pop,Population
1,China,"185,035","+2,742","4,641",+3,"151,407","+1,637","28,987",71,129,3,"160,000,000","111,163","1,439,323,776"
2,USA,"82,383,494","+26,881","1,015,821",+109,"80,244,093","+41,383","1,123,580","1,438","246,299","3,037","996,639,125","2,979,620","334,485,290"
3,India,"43,045,527","+1,247","521,996",+31,"42,511,701",+928,"11,830",698,"30,652",372,"832,104,846","592,521","1,404,347,026"
4,Brazil,"30,261,088","+8,470","662,076",+65,"29,262,483","+35,432","336,529","8,318","140,572","3,076","63,776,166","296,261","215,270,453"
5,France,"27,790,834","+19,810","144,205",+48,"25,132,351","+217,445","2,514,278","1,541","424,076","2,201","260,504,402","3,975,184","65,532,665"


### Bỏ các cột thừa
- Với cột `Deaths/1M pop`, `Tot Cases/1M pop`, `Tests/1M pop`, nhóm sẽ bỏ vì trong nhận thấy những cột này không có tác dụng nhiều trong việc nêu ra mối quan hệ

In [6]:
print(f'no. of cols (prev): {len(list(df))}')
df.drop(columns=['Deaths/1M pop', 'Tot Cases/1M pop', 'Tests/1M pop'], inplace=True)
print(f'no. of cols (now): {len(list(df))}')

no. of cols (prev): 14
no. of cols (now): 11


## Các cột đã đúng kiểu dữ liệu chưa

In [7]:
df.dtypes

Country            object
Total Cases        object
New Cases          object
Total Deaths       object
New Deaths         object
Total Recovered    object
New Recovered      object
Active Cases       object
Serious            object
Total Tests        object
Population         object
dtype: object

- Đổi các cột sau sang kiểu float

In [8]:
df[['New Cases', 'New Deaths', 'New Recovered']] = df[['New Cases', 'New Deaths', 'New Recovered']].applymap( \
                                              lambda x: x if x != x else float(x[1:].replace(',', '')))

In [9]:
cols = list(df)
num_cols = list(set(cols).difference(set(['New Cases', 'New Deaths', 'New Recovered', 'Country'])))
df[num_cols] = df[num_cols].applymap(lambda x: x if x != x else float(x.replace(',', '')))
df.head()

,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,New Recovered,Active Cases,Serious,Total Tests,Population
1,China,185035.0,2742.0,4641.0,3.0,151407.0,1637.0,28987.0,71.0,160000000.0,1.439324e+09
2,USA,82383494.0,26881.0,1015821.0,109.0,80244093.0,41383.0,1123580.0,1438.0,996639125.0,3.344853e+08
3,India,43045527.0,1247.0,521996.0,31.0,42511701.0,928.0,11830.0,698.0,832104846.0,1.404347e+09
4,Brazil,30261088.0,8470.0,662076.0,65.0,29262483.0,35432.0,336529.0,8318.0,63776166.0,2.152705e+08
5,France,27790834.0,19810.0,144205.0,48.0,25132351.0,217445.0,2514278.0,1541.0,260504402.0,6.553266e+07


- Kiểm tra lại kiểu dữ liệu của các thuộc tính

In [10]:
df.dtypes

Country             object
Total Cases        float64
New Cases          float64
Total Deaths       float64
New Deaths         float64
Total Recovered    float64
New Recovered      float64
Active Cases       float64
Serious            float64
Total Tests        float64
Population         float64
dtype: object

$\to$ Các thuộc tính đã đúng kiểu dữ liệu

## Kiểm tra dữ liệu thiếu

In [11]:
def missing_ratio(s):
    return s.isna().sum() * 100 / len(s)

df[cols[1:]].agg([missing_ratio, pd.Series.min, pd.Series.max, pd.Series.nunique])

,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,New Recovered,Active Cases,Serious,Total Tests,Population
missing_ratio,0.0,45.575221,3.982301e+00,69.469027,6.637168e+00,54.424779,6.637168e+00,32.743363,6.194690e+00,0.000000e+00
min,1.0,1.000000,1.000000e+00,1.000000,1.000000e+00,1.000000,0.000000e+00,1.000000,5.117000e+03,8.050000e+02
max,82383494.0,47743.000000,1.015821e+06,213.000000,8.024409e+07,217445.000000,3.238359e+06,8318.000000,9.966391e+08,1.439324e+09
nunique,226.0,104.000000,2.110000e+02,32.000000,2.100000e+02,89.000000,1.940000e+02,87.000000,2.120000e+02,2.260000e+02


$\to$ Tất cả các thuộc tính ít nhiều đều có giá trị thiếu trừ `Total Cases`, `Population`. Ngoài ra các giá trị min, max không có gì bất thường

Chiến lược điền khuyết:
- Với các cột `New Cases`, `New Deaths`, `New Recovered`, `Serious`: không có ca nhiễm mới/ca tử vong mới/ca hồi phục mới/... nào $\to$ Điền số 0
- Với cột `Total Deaths`, nếu ta truy cập vào 1 nước có thuộc tính `Total Deaths` bị thiếu theo đường link có dạng **https://www.worldometers.info/coronavirus/country/{tên-nước}/** (VD: nước Cook Islands thì link là https://www.worldometers.info/coronavirus/country/cook-islands/) thì thuộc tính Deaths (tức `Total Deaths`) = 0 $\to$ Điền số 0 
- Với cột `Total Recovered`, `Total Tests` 
dữ liệu bị thiếu có thể do trang web không thể theo dõi (track) được dữ liệu nước đó nên dẫn tới tình trạng N/A $\to$ Gom nhóm theo châu và lấy median
- Với cột `Active Cases` sẽ tính theo công thức [này](https://www.worldometers.info/), tức Active Cases = (total cases) - (total deaths) - (recovered)

In [12]:
df[['New Cases', 'New Deaths', 'New Recovered', 'Serious', 'Total Deaths']] = \
          df[['New Cases', 'New Deaths', 'New Recovered', 'Serious', 'Total Deaths']].fillna(0)

In [13]:
# Thêm cột `continent`
continent_df = pd.read_csv(f'./continents.csv').rename(columns={'Unnamed: 0': 'Country'})
df = df.merge(continent_df, on='Country', how='inner')

# điền khuyết cho 2 cột 'Total Recovered', 'Total Tests'
mean_filler = df.groupby('continent')[['Total Recovered', 'Total Tests']].transform(lambda x: x.fillna(x.median()))
df[['Total Recovered', 'Total Tests']] = mean_filler[['Total Recovered', 'Total Tests']]
df.head()

,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,New Recovered,Active Cases,Serious,Total Tests,Population,continent
0,China,185035.0,2742.0,4641.0,3.0,151407.0,1637.0,28987.0,71.0,160000000.0,1.439324e+09,Asia
1,USA,82383494.0,26881.0,1015821.0,109.0,80244093.0,41383.0,1123580.0,1438.0,996639125.0,3.344853e+08,North America
2,India,43045527.0,1247.0,521996.0,31.0,42511701.0,928.0,11830.0,698.0,832104846.0,1.404347e+09,Asia
3,Brazil,30261088.0,8470.0,662076.0,65.0,29262483.0,35432.0,336529.0,8318.0,63776166.0,2.152705e+08,South America
4,France,27790834.0,19810.0,144205.0,48.0,25132351.0,217445.0,2514278.0,1541.0,260504402.0,6.553266e+07,Europe


In [14]:
# điền khuyết cho 2 cột 'Active Cases'
fval = df['Total Cases'] - df['Total Deaths'] - df['Total Recovered']
fval[fval < 0] = 0
df['Active Cases'] = df['Active Cases'].fillna(fval)

# Lưu vào file csv mới
- Dữ liệu sau khi tiền xử lý sẽ được lưu với tên table\_{d}\_preprocessed.csv, trong đó {d} là ngày mà nhóm sẽ chọn

In [15]:
df.to_csv(f'./Worldometer-data-preprocessed/table_{d}_preprocessed.csv', index=False)

# Lưu dữ liệu 1 tuần

In [16]:
from google.colab import files
!cp '/content/drive/MyDrive/Colab Notebooks/Lab01/preprocess_pipeline.py' .
import preprocess_pipeline as pp
import os

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
base = f'./Worldometer-data'
df_1_week = pd.DataFrame()

with open('data_1_week.csv', 'w') as f:
  for d in range(18,26,1):
    day = f'{d}-04-2022'
    df = pp.preprocess_pipeline(day)
    df['day'] = day
    df_1_week = pd.concat([df_1_week, df], axis=0)

print(df_1_week.shape)
df_1_week    

(1808, 13)


,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,New Recovered,Active Cases,Serious,Total Tests,Population,continent,day
0,China,185035.0,2742.0,4641.0,3.0,151407.0,1637.0,28987.0,71.0,160000000.0,1.439324e+09,Asia,18-04-2022
1,USA,82383494.0,26881.0,1015821.0,109.0,80244093.0,41383.0,1123580.0,1438.0,996639125.0,3.344853e+08,North America,18-04-2022
2,India,43045527.0,1247.0,521996.0,31.0,42511701.0,928.0,11830.0,698.0,832104846.0,1.404347e+09,Asia,18-04-2022
3,Brazil,30261088.0,8470.0,662076.0,65.0,29262483.0,35432.0,336529.0,8318.0,63776166.0,2.152705e+08,South America,18-04-2022
4,France,27790834.0,19810.0,144205.0,48.0,25132351.0,217445.0,2514278.0,1541.0,260504402.0,6.553266e+07,Europe,18-04-2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...
221,Western Sahara,10.0,0.0,1.0,0.0,9.0,0.0,0.0,0.0,664653.0,6.239160e+05,Africa,25-04-2022
222,Niue,8.0,0.0,0.0,0.0,7.0,0.0,1.0,0.0,76428.5,1.646000e+03,Australia/Oceania,25-04-2022
223,Micronesia,3.0,2.0,0.0,0.0,1.0,0.0,2.0,0.0,76428.5,1.172060e+05,Australia/Oceania,25-04-2022
224,Nauru,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,76428.5,1.094700e+04,Australia/Oceania,25-04-2022


In [ ]:
df_1_week.to_csv(f'./Worldometer-data-preprocessed/data_1_week.csv', index=False)